In [2]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

import timm
model = timm.create_model('convnextv2_base.fcmae_ft_in22k_in1k', pretrained=True)
model = model.eval()

model.eval()
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)
optimized_traced_model = optimize_for_mobile(traced_script_module)
optimized_traced_model._save_for_lite_interpreter("convnextv2_base.fcmae_ft_in22k_in1k_torchscript.pt")

In [6]:
from urllib.request import urlopen
from PIL import Image
import torchvision
img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

# load image into tensor
input_tensor = torchvision.transforms.functional.to_tensor(img).unsqueeze(0)

In [8]:
input_tensor.shape

torch.Size([1, 3, 512, 512])

In [13]:
import time
import torchvision.transforms as transforms

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the saved TorchScript model
loaded_model = torch.jit.load("convnextv2_base.fcmae_ft_in22k_in1k_torchscript.pt")

num_images = 10

with torch.inference_mode():
    start = time.perf_counter()
    for _ in range(num_images):
        output = loaded_model(transform(img).unsqueeze(0))
    end = time.perf_counter()
    time_taken = end - start

average_time = time_taken / num_images
print(f"Total time: {time_taken:.4f} seconds")
print(f"Average time per image: {average_time:.4f} seconds")
print(f"FPS: {1/average_time:.2f}")

Total time: 23.2428 seconds
Average time per image: 2.3243 seconds
FPS: 0.43
